In [50]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import random

In [51]:
DATASET_FOLDER = 'student_resource 3/dataset'
train = pd.read_csv(os.path.join(DATASET_FOLDER, 'train.csv'))
test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))
sample_test = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test.csv'))
sample_test_out = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test_out.csv'))
sample_test_out_fail = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test_out_fail.csv'))

In [52]:
train[:15]

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram
5,https://m.media-amazon.com/images/I/61QsBSE7jg...,731432,item_weight,1400 milligram
6,https://m.media-amazon.com/images/I/81xsq6vf2q...,731432,item_weight,1400 milligram
7,https://m.media-amazon.com/images/I/71DiLRHeZd...,731432,item_weight,1400 milligram
8,https://m.media-amazon.com/images/I/91Cma3Rzse...,731432,item_weight,1400 milligram
9,https://m.media-amazon.com/images/I/71jBLhmTNl...,731432,item_weight,1400 milligram


In [53]:
import re

In [54]:
test.head()

,index,image_link,group_id,entity_name
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth


In [55]:
sample_test.head()

,index,image_link,group_id,entity_name
0,0,https://m.media-amazon.com/images/I/41-NCxNuBx...,658003,width
1,1,https://m.media-amazon.com/images/I/41-NCxNuBx...,658003,depth
2,2,https://m.media-amazon.com/images/I/417NJrPEk+...,939426,maximum_weight_recommendation
3,3,https://m.media-amazon.com/images/I/417SThj+Sr...,276700,voltage
4,4,https://m.media-amazon.com/images/I/417SThj+Sr...,276700,wattage


In [56]:
sample_test_out.head()

,index,prediction
0,0,21.9 foot
1,1,10 foot
2,2,NaN
3,3,289.52 kilovolt
4,4,1078.99 kilowatt


In [57]:
sample_test_out_fail.head()

,index,prediction
0,0,21.9 foot
1,1,10 foot
2,2,NaN
3,3,289.52 kilovolt
4,4,1078.99 kilowatt


In [58]:
unique_entity_names_count = train['entity_name'].nunique()

# Display the unique values and their counts
entity_name_distribution = train['entity_name'].value_counts()

print(f"Number of unique values in 'entity_name': {unique_entity_names_count}")
print("\nDistribution of unique values:")
print(entity_name_distribution)

Number of unique values in 'entity_name': 8

Distribution of unique values:
entity_name
item_weight                      102786
depth                             45127
width                             44183
height                            43597
voltage                            9466
wattage                            7755
item_volume                        7682
maximum_weight_recommendation      3263
Name: count, dtype: int64


In [125]:
import pandas as pd
import random
import requests

# Your OCR function
def extract_text_from_url(api_key, image_url):
    url = 'https://api.ocr.space/parse/image'
    payload = {
        'url': image_url,
        'isOverlayRequired': False,
        'apikey': api_key,
        'language': 'eng'
    }
    
    try:
        response = requests.post(url, data=payload)
        result = response.json()
        
        # Check if result is a dictionary
        if isinstance(result, dict):
            if result.get('IsErroredOnProcessing'):
                print(f"Error for URL {image_url}: {result.get('ErrorMessage')}")
                return None
            else:
                # Extract and return the text
                parsed_results = result.get('ParsedResults', [])
                if parsed_results:
                    return parsed_results[0].get('ParsedText', '')
                else:
                    print(f"No parsed results for URL {image_url}")
                    return None
        else:
            print(f"Unexpected response format for URL {image_url}: {result}")
            return None

    except Exception as e:
        print(f"Exception occurred for URL {image_url}: {e}")
        return None

# Your OCR.Space API key
api_key = 'K86570031188957'

In [104]:
def preprocess_text(text):
    text = text.replace('"', ' inch')
    text = text.replace('/', '\n')
    return text

In [105]:
# Select a random row from the dataset
random_row = train.sample(n=1).iloc[0]

# Extract relevant information from the selected row
random_image_url = random_row['image_link']
group_id = random_row['group_id']
entity_name = random_row['entity_name']
entity_value = random_row['entity_value']

# Extract text from the randomly selected image
extracted_text = extract_text_from_url(api_key, random_image_url)
text = preprocess_text(extracted_text)

# Print the details of the randomly selected image and extracted text
print(f"Random Image URL: {random_image_url}")
print(f"Group ID: {group_id}")
print(f"Entity Name: {entity_name}")
print(f"Expected Entity Value: {entity_value}")
print(f"Extracted Text: {text}")

Random Image URL: https://m.media-amazon.com/images/I/41J98RZVSwS.jpg
Group ID: 120219
Entity Name: width
Expected Entity Value: 15.0 centimetre
Extracted Text: 15cm
5.9in



In [83]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [84]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [85]:
doc = nlp(text)
          
for ent in doc.ents:
    if ent.label_ == "CARDINAL" or ent.label_ == "QUANTITY":
        print(ent.text, "|", ent.label_, "|", spacy.explain(ent.label_))

23.5 inch
20.5 inch | QUANTITY | Measurements, as of weight or distance
30 inch | QUANTITY | Measurements, as of weight or distance
20.5 inch | QUANTITY | Measurements, as of weight or distance


In [86]:
from spacy.pipeline import EntityRuler

ruler = nlp.add_pipe("entity_ruler", before="ner")

In [87]:
patterns = [
    # ITEM_WEIGHT patterns
    {"label": "ITEM_WEIGHT", "pattern": [{"LOWER": "weight"}]},
    {"label": "ITEM_WEIGHT", "pattern": [{"LOWER": "gram"}]},
    {"label": "ITEM_WEIGHT", "pattern": [{"LOWER": "grams"}]},
    {"label": "ITEM_WEIGHT", "pattern": [{"LOWER": "Grams"}]},
    {"label": "ITEM_WEIGHT", "pattern": [{"LOWER": "Gram"}]},
    {"label": "ITEM_WEIGHT", "pattern": [{"LOWER": "mg"}]},
    {"label": "ITEM_WEIGHT", "pattern": [{"LOWER": "milligram"}]},
    {"label": "ITEM_WEIGHT", "pattern": [{"LOWER": "kg"}]},
    {"label": "ITEM_WEIGHT", "pattern": [{"LOWER": "kilogram"}]},
    {"label": "ITEM_WEIGHT", "pattern": [{"LOWER": "ounces"}]},
    {"label": "ITEM_WEIGHT", "pattern": [{"LOWER": "lbs"}]},
    {"label": "ITEM_WEIGHT", "pattern": [{"LOWER": "ton"}]},
    {"label": "ITEM_WEIGHT", "pattern": [{"LOWER": "pound"}]},
    {"label": "ITEM_WEIGHT", "pattern": [{"LOWER": "pounds"}]},
    {"label": "ITEM_WEIGHT", "pattern": [{"LOWER": "KG"}]},
    
    # DEPTH patterns
    {"label": "LWH", "pattern": [{"LOWER": "depth"}]},
    {"label": "LWH", "pattern": [{"LOWER": "centimetre"}]},
    {"label": "LWH", "pattern": [{"LOWER": "cm"}]},
    {"label": "LWH", "pattern": [{"LOWER": "mm"}]},
    {"label": "LWH", "pattern": [{"LOWER": "millimetre"}]},
    {"label": "LWH", "pattern": [{"LOWER": "inch"}]},
    {"label": "LWH", "pattern": [{"LOWER": "in"}]},
    
    # VOLTAGE patterns
    {"label": "VOLTAGE", "pattern": [{"LOWER": "voltage"}]},
    {"label": "VOLTAGE", "pattern": [{"LOWER": "volt"}]},
    {"label": "VOLTAGE", "pattern": [{"LOWER": "v"}]},
    
    # WATTAGE patterns
    {"label": "WATTAGE", "pattern": [{"LOWER": "wattage"}]},
    {"label": "WATTAGE", "pattern": [{"LOWER": "watt"}]},
    {"label": "WATTAGE", "pattern": [{"LOWER": "W"}]},
    
    # ITEM_VOLUME patterns
    {"label": "ITEM_VOLUME", "pattern": [{"LOWER": "volume"}]},
    {"label": "ITEM_VOLUME", "pattern": [{"LOWER": "cup"}]},
    {"label": "ITEM_VOLUME", "pattern": [{"LOWER": "gallon"}]},
    {"label": "ITEM_VOLUME", "pattern": [{"LOWER": "cubic foot"}]},
    {"label": "ITEM_VOLUME", "pattern": [{"LOWER": "fluid ounce"}]},
    {"label": "ITEM_VOLUME", "pattern": [{"LOWER": "millilitre"}]},
    {"label": "ITEM_VOLUME", "pattern": [{"LOWER": "ml"}]},
    {"label": "ITEM_VOLUME", "pattern": [{"LOWER": "litre"}]},
    {"label": "ITEM_VOLUME", "pattern": [{"LOWER": "l"}]},
    {"label": "ITEM_VOLUME", "pattern": [{"LOWER": "cubic inch"}]},
]

In [88]:
ruler.add_patterns(patterns)

In [106]:
doc = nlp(text)
labels_of_interest = {"ITEM_WEIGHT", "LWH", "WIDTH", "HEIGHT", "VOLTAGE", "WATTAGE", "ITEM_VOLUME", "MAX_WEIGHT_RECOMMENDATION"}

# Print only entities that match the defined patterns
for ent in doc.ents:
    if ent.label_ in labels_of_interest:
        start = ent.start_char
        # Extract text immediately before the entity
        preceding_text = text[:start].strip().split('\n')[-1]
        print(f"Value before '{ent.text}': {preceding_text} | Label: {ent.label_}")

Value before 'cm': 15 | Label: LWH
Value before 'in': 5.9 | Label: LWH


# Create Fine Tuned Model

In [119]:
import pandas as pd

grouped = train.groupby('entity_name')

# Sample 10% of each group
sampled_groups = [group.sample(frac=0.10, random_state=1) for name, group in grouped]

# Concatenate the sampled groups into a single DataFrame
sampled_df = pd.concat(sampled_groups)

shuffled_df = sampled_df.sample(frac=1, random_state=1).reset_index(drop=True)

In [120]:
shuffled_df[:15]

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/615-T+HmRQ...,675012,width,10.0 inch
1,https://m.media-amazon.com/images/I/61vwyFbz9k...,788365,item_weight,29.6 gram
2,https://m.media-amazon.com/images/I/610FhGRV+F...,993359,item_weight,100 gram
3,https://m.media-amazon.com/images/I/614UzYCZP2...,478357,height,22.04 inch
4,https://m.media-amazon.com/images/I/61G+XC0OwL...,849273,depth,43.0 centimetre
5,https://m.media-amazon.com/images/I/31Lb54UlNv...,893692,height,19.09 inch
6,https://m.media-amazon.com/images/I/51vNRvOodl...,846116,height,21.0 centimetre
7,https://m.media-amazon.com/images/I/6173JBXwMM...,130202,width,4.53 inch
8,https://m.media-amazon.com/images/I/41NFGijbQV...,452717,height,18.0 centimetre
9,https://m.media-amazon.com/images/I/517WTRfW1R...,311997,height,33.0 centimetre


In [126]:
new_data = []

for index, row in shuffled_df.iterrows():
    image_url = row['image_link'] 
    text = extract_text_from_url(api_key, image_url)
    
    if text:
        new_data.append({
            "text": text,
            "entity_name": row['entity_name'],  
            "entity_value": row['entity_value']  
        })

# Create a new DataFrame with headers
new_train_df = pd.DataFrame(new_data, columns=["text", "entity_name", "entity_value"])

Unexpected response format for URL https://m.media-amazon.com/images/I/615-T+HmRQL.jpg: You may only perform this action upto maximum 180 number of times within 3600 seconds
Unexpected response format for URL https://m.media-amazon.com/images/I/61vwyFbz9kS.jpg: You may only perform this action upto maximum 180 number of times within 3600 seconds
Unexpected response format for URL https://m.media-amazon.com/images/I/610FhGRV+FL.jpg: You may only perform this action upto maximum 180 number of times within 3600 seconds
Unexpected response format for URL https://m.media-amazon.com/images/I/614UzYCZP2L.jpg: You may only perform this action upto maximum 180 number of times within 3600 seconds
Unexpected response format for URL https://m.media-amazon.com/images/I/61G+XC0OwLL.jpg: You may only perform this action upto maximum 180 number of times within 3600 seconds
Unexpected response format for URL https://m.media-amazon.com/images/I/31Lb54UlNvL.jpg: You may only perform this action upto maxi

KeyboardInterrupt: 